# Collision Avoidance - Train Model

Welcome to this host side Jupyter Notebook!  This should look familiar if you ran through the notebooks that run on the robot.  In this notebook we'll train our image classifier to detect two classes
``free`` and ``blocked``, which we'll use for avoiding collisions.  For this, we'll use a popular deep learning library *PyTorch*

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from code_uwyo.uwyo_cnn import collision
from code_uwyo.uwyo_trainer import model_trainer

### Create dataset instance

Now we use the ``ImageFolder`` dataset class available with the ``torchvision.datasets`` package.  We attach transforms from the ``torchvision.transforms`` package to prepare the data for training.  

In [2]:
data_path = 'dataset'
dataset = datasets.ImageFolder(
    data_path,
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

### Split dataset into train and test sets

Next, we split the dataset into *training* and *test* sets.  The test set will be used to verify the accuracy of the model we train.

In [3]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 50, 50])

### Create data loaders to load data in batches

We'll create two ``DataLoader`` instances, which provide utilities for shuffling data, producing *batches* of images, and loading the samples in parallel with multiple workers.

In [4]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=1
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=1
)

### Define the neural network

Finally, we transfer our model for execution on the GPU

In [5]:
model = collision()
device = torch.device('cuda')
model = model.to(device)

### Train the neural network

Using the code below we will train the neural network for 30 epochs, saving the best performing model after each epoch.

> An epoch is a full run through our data.

In [6]:
keys = ['Loss', 'Accuracy']
epochs = 10
model_path = 'best_model_cnn.pth'
best_accuracy = 0.0

# Declare optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
loss = torch.nn.CrossEntropyLoss()

trainer = model_trainer(device, model, epochs, loss, optimizer, keys)

for epoch in range(epochs):
    trainer.Clear_Metrics()
    trainer.train(train_loader)
    trainer.test(test_loader)
    trainer.Print_Metrics()
    
    test_accuracy = trainer.metrics['Testing Accuracy']
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), model_path)
        best_accuracy = test_accuracy
        
print('Training Complete') 

RuntimeError: CUDA error: device-side assert triggered

Once that is finished, you should see a file ``best_model_cnn.pth`` in the Jupyter Lab file browser.  Select ``Right click`` -> ``Download`` to download the model to your workstation

In [ ]:
trainer.Plot('Accuracy')

In [ ]:
trainer.Plot('Loss')